In [8]:
import glob
import email
import re
from email import policy
from email.parser import BytesParser

In [32]:
def email_parser():
    files = glob.glob("DATA\*.eml")
    text_emails = set()
    for file in files:
        with open(file, 'rb') as fp:
            msg = BytesParser(policy=policy.default).parse(fp)
            try:
                text = msg.get_body(preferencelist=('plain')).get_content()
            except:
                text.encode('utf-8', 'surrogateescape').decode('utf-8') 
            text_emails.add(text)
    return text_emails        

def label_parser():
    file = open("SPAMTrain.label", "r")
    if file.mode == 'r':
        return file.read()    

In [33]:
emails = email_parser()
labels = label_parser()

In [46]:
class NaiveBayes:
    def __init__(self, k):
        self.k = k
    def create_training_set(self, emails, labels):
        words_set = dict()
        labels_set = [0,0] 
        for text in emails:
            email_words = re.split(" |\n", text)
            for word in email_words:
                try:
                    words_set[word] += 1
                except:
                    words_set[word] = 1
        labels_set[0] = labels.count('0')
        labels_set[1] = labels.count('1')
        return words_set, labels_set

In [47]:
nb = NaiveBayes(1)

In [48]:
nb.create_training_set(emails, labels)

({'Custom': 16,
  'Warez': 2,
  'CDsFrom': 1,
  'nobody': 373,
  'Fri': 335,
  'Feb': 341,
  '14': 392,
  '11:52:50': 53,
  '2014': 329,
  'Content-Type:': 366,
  'text/html;': 42,
  'charset="iso-8859-1"': 40,
  'Content-Transfer-Encoding:': 324,
  '7bit': 80,
  '': 526299,
  '<TABLE': 300,
  'cellSpacing=0': 161,
  'cellPadding=0': 123,
  'width=725': 1,
  'border=0>': 313,
  '<TBODY>': 218,
  '<TR>': 502,
  '<TD': 774,
  'style="FONT-SIZE:': 86,
  '8pt;': 70,
  'FONT-FAMILY:': 134,
  'Tahoma"': 22,
  'vAlign=top': 87,
  'width=470': 2,
  'rowSpan=2>': 2,
  'height=111': 2,
  'width=723': 3,
  'colSpan=4': 1,
  'height=111><B><FONT': 1,
  'color=#ff6600>Introduction</FONT></B>': 1,
  '<BR>We': 2,
  'sell': 141,
  'Backup': 19,
  'CDs,': 9,
  'also': 1321,
  'known': 235,
  'as': 4836,
  'CDs.': 6,
  'CDs': 41,
  'are': 5371,
  'copies': 38,
  'of': 22203,
  'Software.': 4,
  '<P>For': 1,
  'example': 172,
  'if': 2150,
  'you': 8200,
  'go': 654,
  'into': 1116,
  'a': 18741,
  'shop